DiffDIS
====

**High-Precision Dichotomous Image Segmentation via Probing Diffusion Capacity**

 * Paper: https://arxiv.org/abs/2410.10105

![DiffDIS Overview](../assets/diffdis_overview.png)



! Installation

```bash
git clone https://github.com/qianyu-dlut/DiffDIS.git
cd DiffDIS

pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118
pip install -r requirements.txt
pip install -e diffusers-0.30.2/
```